In [6]:
# 1. 필요한 라이브러리들을 모두 import 합니다.
import numpy as np
from stable_baselines3 import DQN
from emg_env_bu import emg_env  # 이전에 만든 강화학습 환경
from cnn_feature_extractor import CNN_feature # 직접 만든 CNN 특징 추출기

print("라이브러리 로딩 완료!")

# 2. 파일 경로 및 주요 변수들을 한 곳에서 관리합니다.
#    경로가 다르다면 이 부분만 수정하면 됩니다.
TRAIN_DATA_DIR = 'リアルタイム/spec_tensor/'
TEST_DATA_DIR = 'data/test/'
MODEL_SAVE_PATH = "dqn_emg_final_model.zip" # 저장될 모델의 전체 파일 이름

print(f"훈련 데이터 경로: {TRAIN_DATA_DIR}")
print(f"테스트 데이터 경로: {TEST_DATA_DIR}")
print(f"모델 저장 경로: {MODEL_SAVE_PATH}")

라이브러리 로딩 완료!
훈련 데이터 경로: リアルタイム/spec_tensor/
테스트 데이터 경로: data/test/
모델 저장 경로: dqn_emg_final_model.zip


In [ ]:
# --- [셀 2: 모델 훈련 및 저장] ---

# 1. 훈련용 환경을 생성합니다. (셀 1에서 정의한 변수 사용)
train_env = emg_env(data_dir=TRAIN_DATA_DIR, train=True)

# 2. 우리가 직접 만든 CNN 특징 추출기를 사용하도록 설정합니다.
policy_kwargs = dict(
    features_extractor_class=CNN_feature,
    features_extractor_kwargs=dict(features_dim=256),  # 최종 특징 벡터의 크기
)

# 3. DQN 모델을 생성합니다.
model = DQN(
    'CnnPolicy',
    train_env,
    policy_kwargs=policy_kwargs,
    buffer_size=50000,       # 리플레이 버퍼 크기
    learning_starts=1000,    # 이 스텝만큼 경험을 쌓은 후 학습 시작
    batch_size=64,           # 한 번에 학습할 데이터의 양
    learning_rate=1e-4,      # 학습률
    gamma=0.99,              # 미래 보상에 대한 할인 계수
    tau=1.0,                 # 타겟 네트워크 업데이트 강도 (1.0은 하드 업데이트)
    train_freq=4,            # 4 스텝마다 훈련
    gradient_steps=1,        # 각 훈련마다 그래디언트 업데이트 1번 수행
    verbose=1,               # 훈련 과정에 대한 로그 출력
    tensorboard_log="./logs/dqn_emg_custom/" # 텐서보드 로그 저장 경로
)

# 4. 모델 훈련을 시작합니다.
print("===== 모델 훈련 시작 =====")
model.learn(total_timesteps=42000, log_interval=4)
print("===== 모델 훈련 완료 =====")


# 5. 훈련된 모델을 파일로 저장합니다. (셀 1에서 정의한 변수 사용)
model.save(MODEL_SAVE_PATH)
print(f"모델이 '{MODEL_SAVE_PATH}' 경로에 성공적으로 저장되었습니다.")

# 훈련 환경 종료
train_env.close()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
===== 学習開始 =====
Logging to ./logs/dqn_emg_custom/DQN_7
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 113      |
|    ep_rew_mean      | -78.5    |
|    exploration_rate | 0.898    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 164      |
|    time_elapsed     | 2        |
|    total_timesteps  | 452      |
----------------------------------
모든 데이터를 한 번씩 사용했습니다. 처음부터 다시 시작합니다.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 113      |
|    ep_rew_mean      | -78.2    |
|    exploration_rate | 0.796    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 156      |
|    time_elapsed     | 5        |
|    total_timesteps  | 904      |
----------------------------------
----------------------------------
| rollout

In [10]:
# 1. 파일에 저장된 모델을 불러옵니다. (셀 1에서 정의한 변수 사용)
print(f"'{MODEL_SAVE_PATH}' 경로에서 모델을 불러옵니다...")
loaded_model = DQN.load(MODEL_SAVE_PATH)
print("모델 로딩 완료!")

# 2. 테스트용 환경을 생성합니다. (셀 1에서 정의한 변수 사용)
test_env = emg_env(data_dir=TEST_DATA_DIR, train=False)
obs, info = test_env.reset()
done = False
total_reward = 0

print("\n===== 모델 테스트 시작 =====")
# 에피소드가 끝날 때까지 반복
while not done:
    # 모델을 사용해 현재 관측(obs)에 대한 행동(action)을 예측
    action, _ = loaded_model.predict(obs, deterministic=True)
    
    # 예측한 행동을 환경에 전달하고 다음 상태, 보상 등을 받음
    obs, reward, terminated, truncated, info = test_env.step(action)
    
    # 종료 조건 확인
    done = terminated or truncated
    
    # 총 보상 합산
    total_reward += reward
    
    # 예측 결과와 실제 정답, 보상을 출력
    # action은 numpy 배열이므로 .item()을 사용해 순수한 숫자로 변환
    print(f"예측 행동: {action.item()}, 실제 레이블: {test_env.episode_label}, 이번 스텝 보상: {reward}")

print("===== 모델 테스트 완료 =====")
print(f"에피소드 총 보상: {total_reward}")

# 테스트 환경 종료
test_env.close()

'dqn_emg_final_model.zip' 경로에서 모델을 불러옵니다...
모델 로딩 완료!

===== 모델 테스트 시작 =====
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 레이블: 0, 이번 스텝 보상: 1.0
예측 행동: 0, 실제 